# Natural frequency of a traffic signal 🚏

The purpose of this notebook is to find an approximate value of the frequency of a traffic signal. The Rayleigh's method will be used to reduce the continuous structure into a single degree of freedom system.

## Geometry definitions

Before solving the Rayleigh integrals, some auxiliary functions can be defined to calculate the structural properties -linear density, inertia, point mass- from the geometric definition.

To avoid conversion errors, all the units are in *international system*.

In [ ]:
from math import pi, sqrt

steel_density = 7700     # kg/m3
young_modulus = 210*1e9  # N/m2

def CircularSignal(diameter, thickness):                        # Input dimensions in m
    return steel_density * thickness * pi * 0.25 * diameter**2  # kg

def SquareSignal(width, height, thickness):            # Input dimensions in m
    return steel_density * thickness * width * height  # kg

def CircularCrossSection(diameter, thickness):                                  # Input dimensions in m
    linear_density = steel_density * thickness * pi * diameter                  # kg/m
    EI = young_modulus * 1/64 * pi * (diameter**4 - (diameter-2*thickness)**4)  # N*m2
    return linear_density, EI

def RectangularCrossSection(width, depth, thickness):                                          # Input dimensions in m
    linear_density = steel_density * thickness * 2 * (width+depth)                             # kg/m
    EI = young_modulus * 1/12 * (width*depth**3 - (width-2*thickness)*(depth-2*thickness)**3)  # N*m2
    return linear_density, EI

 ## Rayleigh's method

The traffic signal is a structure clamped at the bottom and free at the top. Hence, a second order polynomial is choosen as shape function:
$$
\psi = \frac{z^2}{h^2} \\
\psi´´ = \frac{2}{h^2}
$$
which satisfies the kynematic boundary conditions $\psi(0)=\psi'(0)=0$.

The frequency is obtained after some algebra using the Rayleigh's expressions:
\begin{align*}
m &= \int_0^h \rho A\psi^2 dz + \sum \psi_i^2 m_i = \frac{1}{5} \rho A h + m_1 \\
k &= {\displaystyle \int_0^h EI\psi''^2 dz} = 4 \frac{EI}{h^3} \\
\end{align*}

In [ ]:
def NaturalFrequencyPoly(EI, lin_density, top_mass, height):
    m = lin_density * height / 5 + top_mass
    k = 4 * EI / height**3
    frequency = sqrt(k/m)           # rad/s
    nat_frequency = frequency/2/pi  # Hz
    return nat_frequency

### Trigonometric shape function

$$
\psi = 1 - \cos(2\pi z/h) \\
\psi´´ = \frac{4\pi^2}{h^2}\cos(2\pi z/h)
$$

In [ ]:
def NaturalFrequencyTrig(EI, lin_density, top_mass, height):
    m = lin_density * height * (3/2 -4/pi) + top_mass
    k = EI * pi**4 / height**3 / 32
    frequency = sqrt(k/m)           # rad/s
    nat_frequency = frequency/2/pi  # Hz
    return nat_frequency

## Calculations

In [ ]:
rho1, EI1 = RectangularCrossSection(width=.080, depth=.040, thickness=.001)
signal_mass1 = CircularSignal(diameter=.600, thickness=.001)

nat_frequency = NaturalFrequencyPoly(EI1, rho1, signal_mass1, height=3.5)

print(f'Natural frequency: {nat_frequency:.1f} Hz')

In [ ]:
rho2, EI2 = CircularCrossSection(diameter=.060, thickness=.001)
signal_mass2 = CircularSignal(diameter=.650, thickness=.001)
signal_mass2 += SquareSignal(width=.600, height=.800, thickness=.001)

nat_frequency = NaturalFrequencyPoly(EI2, rho2, signal_mass2, height=3)

print(f'Natural frequency: {nat_frequency:.1f} Hz')

In [ ]:
nat_frequency = NaturalFrequencyTrig(EI1, rho1, signal_mass1, height=3)

print(f'Natural frequency: {nat_frequency:.1f} Hz')